<a href="https://colab.research.google.com/github/mangusta-n/miniprojects/blob/main/tree_describer_chop_and_replant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Загрузка и предобработка файла

In [55]:
import pandas as pd
import numpy as np

In [2]:
pd.options.mode.chained_assignment = None

https://drive.google.com/file/d/1Cwu7daDJ1tn9tHK7lSTP4R6GBVUynMVQ/view?usp=sharing

In [8]:
! gdown --id 1Cwu7daDJ1tn9tHK7lSTP4R6GBVUynMVQ


/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1Cwu7daDJ1tn9tHK7lSTP4R6GBVUynMVQ
To: /content/perechetka_per.csv
100% 29.9k/29.9k [00:00<00:00, 24.9MB/s]


In [9]:
df = pd.read_csv('/content/perechetka_per.csv', sep=';')

In [10]:
df.head()

,number,type,tree,bush,diameter,height,description,action
0,1,тополь (поросль),NaN,1.0,NaN,2,поросль,Вырубить
1,2,Самосев до 8 см.,1.0,NaN,6,3,клен ясенелистный,Вырубить
2,3,Самосев до 8 см.,1.0,NaN,6,3,ива,Вырубить
3,4,Береза,1.0,NaN,16,12,удовл.,Вырубить
4,5,клен ясенелистный (поросль),NaN,6.0,NaN,2,поросль,Вырубить


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 313 entries, 0 to 312
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   number       313 non-null    int64  
 1   type         313 non-null    object 
 2   tree         287 non-null    float64
 3   bush         26 non-null     float64
 4   diameter     287 non-null    object 
 5   height       313 non-null    object 
 6   description  313 non-null    object 
 7   action       313 non-null    object 
dtypes: float64(2), int64(1), object(5)
memory usage: 19.7+ KB


В столбцах tree и bush есть пропуски - это специфика данных, в tree отмечено количество только деревьев, bush соответственно кустов

In [12]:
df.tree.value_counts()

1.0     252
2.0      13
3.0       6
20.0      2
35.0      2
5.0       2
4.0       2
15.0      2
7.0       1
25.0      1
8.0       1
10.0      1
16.0      1
9.0       1
Name: tree, dtype: int64

In [13]:
print(f'Общее количество деревьев {df.tree.sum()} и кустарников {df.bush.sum()}')
df.groupby('action', as_index=False)[['tree', 'bush']].sum()

Общее количество деревьев 529.0 и кустарников 466.0


,action,tree,bush
0,Вырубить,474.0,466.0
1,Пересадить,39.0,0.0
2,Сохранить,16.0,0.0


##Получение данных

Для расчетов по вырубке нам нужны следующие данные: 
1. количество поросли и кустарников, 
2. количество деревьев твердолиственных и мягколиственных, 
3. количество деревьев по группам (диаметр от 1 до 10, от 11 до 20 и тд), 
4. является ли дерево многоствольным, число стволов, 
5. диаметр пня. Если указно несколько диаметров, берем бОльший. Диаметр пня у многоствольных больше. 
6. Количество вывозимых порубочных остатков

###Вырубка. Кустарники

In [14]:
def chop_bush_counter(df):
  try:
    df_bush = df[(~df.bush.isna()) & (df.action == 'Вырубить')]\
          [['type', 'bush', 'height', 'description']]
    df_bush['bush_cat'] = df['type'].apply(lambda x: '1поросль' if x == 'поросль' else '2куст')
    df_bush_to_chop = df_bush.groupby('bush_cat', as_index=False)[['bush']].sum()
    return df_bush_to_chop
  except:
    print('Ошибочка')

In [15]:
df_bush_to_chop = chop_bush_counter(df)

In [16]:
df_bush_to_chop

,bush_cat,bush
0,1поросль,2.0
1,2куст,464.0


In [17]:
bush_weigth = df_bush_to_chop.bush[0]*0.05 + df_bush_to_chop.bush[1]*0.1

In [19]:
print(f'Общее количество кустов {df_bush_to_chop.bush.sum()}, вес порубочных остатков {round(bush_weigth,2)} т.')

Общее количество кустов 466.0, вес порубочных остатков 46.5 т.


###Вырубка. Деревья

Сначала выделим некоторые вспомогательные функции: деление на группы, подсчет ствольности, деление на категории по твердости древесины, и впомогательные данные по весу в тоннах относительно диаметра ствола

In [20]:
#ствольность обычно указана в комментарии
def is_mnogostv(row):
  for i in range(6):
    if str(i) in row:
        return int(i)
  return 1   

In [21]:
def tree_type(row):
  tverd_types = ['Клен', 'Бук', 'Орех', 'Ясень' , 'Яблоня', 'Вишня', 
           'Сухостой', 'Самосев', 'Рябина', 'Акация']
  for tree_type in tverd_types:
    if tree_type.lower() in row.lower():
      return 'Твердолиственное'
  return 'Мягколиственное/хвойное'

In [22]:
def diameter_bins(row):
  return f'до {row // 10 + 1}0'

In [23]:
df_mass = {'до 10' : 0.0794285714285714, 
           'до 20' : 0.277714285714286, 
           'до 30' : 0.763809523809524, 
           'до 40' : 1.53942857142857, 
           'до 50' : 2.67476190476191, 
           'до 60' : 4.01542857142857, 
           'до 70' : 6.57371428571429, 
           'до 80' : 8.21714285714286, 
           'до 90' : 10.6822857142857, 
           'поросль' : 0.05, 
           'куст' : 0.01}

А теперь собственно функция, которая посчитает нам итоговую таблицу c количеством деревьев по категориям и диаметром и количеством порубочных остатков (а также диаметры пней для корчевки).

In [24]:
def chop_tree_counter(df):
  try:
    df_tree = df[(df.bush.isna()) & (df.action == 'Вырубить')]\
              [['type', 'tree', 'diameter', 'height', 'description']]
    df_tree['stvol'] = df.description.apply(is_mnogostv)
    df_tree['diameter'] = df_tree['diameter'].apply(lambda x: int(x.split('-')[-1]))
    df_tree['stvol_count'] = df_tree['stvol'] * df_tree['tree']
    df_tree['stump_diameter'] = df_tree['diameter'] * df_tree['stvol'] + 5
    df_tree['tree_type'] = df_tree['type'].apply(tree_type)
    df_tree['diameter_bins'] = df_tree['diameter'].apply(diameter_bins)
    df_tree['stump_diameter_bins'] = df_tree['stump_diameter'].apply(diameter_bins)
    df_tree['weight'] = df_tree['diameter_bins'].map(df_mass) * df_tree['stvol_count']
    df_tree_grouped = df_tree.groupby(['tree_type', 'diameter_bins'])\
              [['tree', 'stvol_count', 'weight']].sum()
    df_tree_trees = (pd.concat([df_tree_grouped,
           df_tree_grouped.groupby(level=0).sum()
           .assign(claim_type= "total")
           .set_index('claim_type', append=True)]).sort_index()
           .rename(columns={'tree':'Кол-во деревьев', 
                            'stvol_count': 'Кол-во стволов',
                            'weight': 'Тоннаж'}))
    df_stumps = (df_tree.groupby('stump_diameter_bins')[['tree']].sum()
                .rename(columns={'tree':'Кол-во пней'}))
    return df_tree_trees, df_stumps
  except:
    print('Ошибочка')

In [25]:
trees, stumps = chop_tree_counter(df)

In [26]:
trees

Кол-во деревьев  Кол-во стволов  \
tree_type               diameter_bins                                    
Мягколиственное/хвойное total                     69.0            68.0   
                        до 10                      9.0             6.0   
                        до 20                     44.0            44.0   
                        до 30                     13.0            15.0   
                        до 40                      3.0             3.0   
Твердолиственное        total                    405.0           472.0   
                        до 10                    203.0           211.0   
                        до 20                    145.0           180.0   
                        до 30                     47.0            58.0   
                        до 40                      9.0            20.0   
                        до 50                      1.0             3.0   

                                           Тоннаж  
tree_type               diameter_bins              
Мягколиственное/хвойное total           28.771429  
                        до 10            0.476571  
                        до 20           12.219429  
                        до 30           11.457143  
                        до 40            4.618286  
Твердолиственное        total          149.861810  
                        до 10           16.759429  
                        до 20           49.988571  
                        до 30           44.300952  
                        до 40           30.788571  
                        до 50            8.024286

In [27]:
tree_weigh = trees.query('diameter_bins == "total"')['Тоннаж'].sum()

In [29]:
print(f'''Итого вывозимый тоннаж порубочных остатков 
      {tree_weigh.round(2)} от деревьев, 
      {round(bush_weigth, 2)} от кустов, 
      всего {round(tree_weigh + bush_weigth, 2)} тонны''')

Итого вывозимый тоннаж порубочных остатков 
      178.63 от деревьев, 
      46.5 от кустов, 
      всего 225.13 тонны


In [30]:
stumps

,Кол-во пней
stump_diameter_bins,
до 10,12.0
до 100,1.0
до 130,2.0
до 140,1.0
до 160,1.0
до 20,296.0
до 30,113.0
до 40,27.0
до 50,6.0


###Пересадка

Для расчета сметы на пересадку нам нужно количество деревьев и кустарников в разрезе комов (обычно указаны в комментариях, но могут быть и не указаны). Поэтому на всякий случай считаем ком по правилам, какой он должен быть, а потом сравним то, что есть в перечетке и то, как надо, и принимаем решения, какие данные использовать.

In [64]:
#размер кома по диаметру ствола
def kom_size(row):
  try:
    if np.isnan(row):
      return '0,5x0,5x0,4'
    if row < 5:
      return '0,8x0,8x0,45'
    if row == 5:
      return '1,0х1,0х0,6'
    if row == 6:
      return '1,3х1,3х0,65'
    if row >= 7 and row <= 9:
      return '1,5х1,5х0,65' 
    if  row >= 10 and row <= 12:
      return '1,7х1,7х0,7' 
    if  row >= 13 and row <= 15:
      return '2,0х2,0х0,8' 
    if  row >= 16:
      return '2,4х2,4х0,85' 
  except:
    print('Проверьте данные')

In [65]:
def replant_tree_counter(df):
  try:
    df_per = pd.concat([df[(df.bush.isna()) & (df.action == 'Пересадить')]
                        [['tree', 'diameter', 'description']],
              df[(df.tree.isna()) & (df.action == 'Пересадить')]
                        [['bush', 'diameter', 'description']]])
    df_per['diameter'] = pd.to_numeric(df_per['diameter'])
    df_per['new_kom'] = df_per['diameter'].apply(kom_size)
    df_per_kom = df_per.groupby('description')[['tree', 'bush']].sum()
    df_per_new_kom = df_per.groupby('new_kom')[['tree', 'bush']].sum()
    return df_per_kom, df_per_new_kom
  except:
    print('Ошибочка')

In [66]:
old_kom, new_kom = replant_tree_counter(df)

In [67]:
old_kom

,tree,bush
description,,
"1,0х1,0х0,6",2.0,0.0
"1,5х1,5х0,65",37.0,0.0


In [68]:
new_kom

,tree,bush
new_kom,,
"0,8x0,8x0,45",2.0,0.0
"1,3х1,3х0,65",37.0,0.0
